In [1]:
# импортирование библиотек
import pandas as pd
import numpy as np
import re
from datetime import datetime

# константы
COSTS = 'm_costs'
CLICKS = 'm_clicks'
DATE = 'created_at'
KEYS = ['created_at',
        'd_utm_source',
        'd_utm_medium',
        'd_utm_campaign',
        'd_utm_content']

# Обзор данных

In [2]:
# загрузка данных
path = '' # укажите путь к файлу данных

ads = pd.read_csv(path + 'ads.csv')
leads = pd.read_csv(path + 'leads.csv')
purchases = pd.read_csv(path + 'purchases.csv')

In [3]:
def show_review(data):
    display(data.head())
    display(data.tail())
    display(data.info())
    print(f'Количество явных дубликатов: {data.duplicated().sum()}')

In [4]:
show_review(ads)

,created_at,d_ad_account_id,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,d_utm_term,m_clicks,m_cost
0,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8404700756,NaN,2.0,48.840
1,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476161,NaN,3.0,100.776
2,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476162,NaN,1.0,10.140
3,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476163,NaN,0.0,0.000
4,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476164,NaN,0.0,0.000


,created_at,d_ad_account_id,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,d_utm_term,m_clicks,m_cost
8597,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477628,NaN,0.0,0.0
8598,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477629,NaN,0.0,0.0
8599,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8404708323,NaN,0.0,0.0
8600,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8404708324,NaN,0.0,0.0
8601,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477631,NaN,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8602 entries, 0 to 8601
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   created_at       8602 non-null   object 
 1   d_ad_account_id  8602 non-null   object 
 2   d_utm_source     8602 non-null   object 
 3   d_utm_medium     8602 non-null   object 
 4   d_utm_campaign   8602 non-null   int64  
 5   d_utm_content    8602 non-null   int64  
 6   d_utm_term       0 non-null      float64
 7   m_clicks         8602 non-null   float64
 8   m_cost           8602 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 605.0+ KB


None

Количество явных дубликатов: 0


In [5]:
show_review(leads)

,lead_created_at,lead_id,d_lead_utm_source,d_lead_utm_medium,d_lead_utm_campaign,d_lead_utm_content,d_lead_utm_term,client_id
0,2022-10-16,d8c4a9e8-4d4a-11ed-907e-848f69e142cf,NaN,NaN,NaN,NaN,NaN,d8c4a9e9-4d4a-11ed-907e-848f69e142cf
1,2022-10-16,c48f9767-4d4a-11ed-907e-848f69e142cf,NaN,NaN,NaN,NaN,NaN,c48f9768-4d4a-11ed-907e-848f69e142cf
2,2022-10-16,d74ac455-4d49-11ed-907e-848f69e142cf,vkontakte,social,kartasetevaya,NaN,NaN,f64c97d5-31ff-11ea-abeb-c412f533dba1
3,2022-10-16,0f69e77d-4d46-11ed-907e-848f69e142cf,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-16,620ee1eb-4d45-11ed-907e-848f69e142cf,ycard,social,all,NaN,NaN,NaN


,lead_created_at,lead_id,d_lead_utm_source,d_lead_utm_medium,d_lead_utm_campaign,d_lead_utm_content,d_lead_utm_term,client_id
23535,2022-09-07,793a94f2-2e88-11ed-907b-848f69e142cf,NaN,NaN,NaN,NaN,NaN,793a94f3-2e88-11ed-907b-848f69e142cf
23536,2022-09-07,56bb2186-2e86-11ed-907b-848f69e142cf,sms,email,0509_01,NaN,NaN,09098204-53c3-11ea-abeb-c412f533dba1
23537,2022-10-16,c045f5aa-4d5e-11ed-907e-848f69e142cf,NaN,NaN,NaN,NaN,NaN,d0f407cc-4d5e-11ed-907e-848f69e142cf
23538,2022-10-16,a521de75-4d59-11ed-907e-848f69e142cf,NaN,NaN,NaN,NaN,NaN,NaN
23539,2022-10-16,818a8ae3-4d58-11ed-907e-848f69e142cf,kviz,cpc,2,NaN,NaN,9473caee-4d59-11ed-907e-848f69e142cf


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23540 entries, 0 to 23539
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   lead_created_at      23540 non-null  object
 1   lead_id              23540 non-null  object
 2   d_lead_utm_source    12331 non-null  object
 3   d_lead_utm_medium    10100 non-null  object
 4   d_lead_utm_campaign  9569 non-null   object
 5   d_lead_utm_content   3338 non-null   object
 6   d_lead_utm_term      328 non-null    object
 7   client_id            18601 non-null  object
dtypes: object(8)
memory usage: 1.4+ MB


None

Количество явных дубликатов: 0


In [6]:
show_review(purchases)

,purchase_created_at,purchase_id,client_id,m_purchase_amount
0,2022-02-21,d3198d39-6d16-40c9-bff7-aa28bd6e2991,7011bdcd-6fd8-11e7-80fc-c412f533dba1,9950.0
1,2022-02-21,48a0ad24-77aa-4064-a971-dd0d6f1f6c50,a2771bb0-6fd6-11e7-80fc-c412f533dba1,8700.0
2,2022-02-21,267ff20f-a56c-480a-b2f7-1f7b1a746f9f,2687f503-6fd7-11e7-80fc-c412f533dba1,0.0
3,2022-02-21,19fba5f0-785d-45bd-903a-34ba529c6404,01122a97-45bc-11eb-ac23-c412f533dba1,8700.0
4,2022-02-21,00ac921d-32e3-4200-9435-a46d5fef4a29,a55b8c4e-d5a3-11e9-abbe-c412f533dba1,500.0


,purchase_created_at,purchase_id,client_id,m_purchase_amount
66175,2022-02-19,8c1cdf21-2c6a-4ae0-8de4-8c86b6a217a5,d1f7b6d1-644f-11ea-abf3-c412f533dba1,110.0
66176,2022-02-19,161adaa3-419d-4c9e-bbab-ad9d32caaf0c,41ec1281-8232-11eb-ac27-c412f533dba1,119.0
66177,2022-08-31,db2b1f43-9963-44a4-b81f-a35d086250fa,85b0f39c-0c60-11eb-ac1a-c412f533dba1,9450.0
66178,2022-08-31,35cd2dd2-dc7c-4245-a28d-de3c352aef7e,b04dc805-0e32-11ec-9043-848f69e142cf,6495.0
66179,2022-08-31,92d9e347-bed4-42f4-8240-729d50e26073,f1e3a10b-1540-11eb-ac1d-c412f533dba1,12150.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66180 entries, 0 to 66179
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   purchase_created_at  66180 non-null  object 
 1   purchase_id          66180 non-null  object 
 2   client_id            66179 non-null  object 
 3   m_purchase_amount    66180 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


None

Количество явных дубликатов: 0


Описание данных:

1. **`ads` (рекламные объявления)**:
    - `created_at` - дата рекламного объявления
    - `d_ad_account_id` - id рекламного объявления
    - `d_utm_source` - utm метка
    - `d_utm_medium` - utm метка
    - `d_utm_campaign` - utm метка
    - `d_utm_content` - utm метка
    - `d_utm_term` - utm метка
    - `m_clicks` - количество кликов (общее / среднее ?)
    - `m_cost` - стоимость (общее / среднее ?)

2. **`leads` (лиды)**:
    - `lead_created_at` - дата создания лида
    - `lead_id` - id лида
    - `d_lead_utm_source` - utm метка
    - `d_lead_utm_medium` - utm метка
    - `d_lead_utm_campaign` - utm метка
    - `d_lead_utm_content` - utm метка
    - `d_lead_utm_term` - utm метка
    - `client_id` - id клиента

3. **`purchases` (покупки)**:
    - `purchase_created_at` - дата покупки
    - `purchase_id` - id покупки
    - `client_id` - id клиента
    - `m_purchase_amount` - стоимость покупки

In [7]:
display(ads.created_at.min())
ads.created_at.max()

'2022-01-03'

'2022-09-14'

In [8]:
display(leads.lead_created_at.min())
leads.lead_created_at.max()

'2022-01-01'

'2022-10-16'

## Вывод

1. Данные рекламы представлены с `2022-01-03` по `2022-09-14`, при этом у части лидов дата раньше первой рекламы (2022-01-01 и 2022-01-02). Исключим данное логическое несоответствие.
2. В таблице рекламы пропущены значения `d_utm_term` - в дальнейшем можно отобрать только те лиды, для которых данных отметок также не будет, а столбец удалить в обеих таблицах. Об особенностях получения данной метки необходимо проконсультироваться с профильным специалистом.
3. В данных содержится множество других `пропусков` - заполним их там, где это возможно.
4. Типы данных `utm меток` различаются - для последующего объединения таблиц необходимо привести их к одному типу. Также для объединения таблиц переименуем столбцы.

# Предобработка данных

In [9]:
# обработка логических несоответствий
leads = leads[leads.lead_created_at >= ads.created_at.min()]

In [10]:
# обработка неинформативного столбца 'd_utm_term'
ads = ads.drop('d_utm_term', axis=1).reset_index(drop=True)
leads = leads[leads.d_lead_utm_term.isna()].drop('d_lead_utm_term', axis=1).reset_index(drop=True)

In [11]:
# приведение данных к одному типу
ads[['d_utm_campaign','d_utm_content']] = \
ads[['d_utm_campaign','d_utm_content']].astype(str)

In [12]:
# обработка пропущенных значений
leads = leads.fillna('unknown')
purchases.client_id = purchases.client_id.fillna('unknonw_client')

# Объединение таблиц

In [13]:
# унификация названий столбцов для объединения таблиц
for old_col in leads.columns:
    new_col = re.sub('_lead', '', old_col)
    leads = leads.rename(columns={old_col: new_col})
leads = leads.rename(columns={'lead_created_at': DATE})

In [14]:
# объединение таблиц
merge_ad_lead = ads.merge(leads,
                          how='left',
                          on=KEYS)

merge_ad_lead_client = merge_ad_lead.merge(purchases,
                                           how='left',
                                           on='client_id')

In [15]:
show_review(merge_ad_lead_client)

,created_at,d_ad_account_id,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,m_clicks,m_cost,lead_id,client_id,purchase_created_at,purchase_id,m_purchase_amount
0,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8404700756,2.0,48.840,NaN,NaN,NaN,NaN,NaN
1,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476161,3.0,100.776,NaN,NaN,NaN,NaN,NaN
2,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476162,1.0,10.140,NaN,NaN,NaN,NaN,NaN
3,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476163,0.0,0.000,NaN,NaN,NaN,NaN,NaN
4,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476164,0.0,0.000,NaN,NaN,NaN,NaN,NaN


,created_at,d_ad_account_id,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,m_clicks,m_cost,lead_id,client_id,purchase_created_at,purchase_id,m_purchase_amount
10050,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477628,0.0,0.0,NaN,NaN,NaN,NaN,NaN
10051,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477629,0.0,0.0,NaN,NaN,NaN,NaN,NaN
10052,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8404708323,0.0,0.0,NaN,NaN,NaN,NaN,NaN
10053,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8404708324,0.0,0.0,NaN,NaN,NaN,NaN,NaN
10054,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477631,0.0,0.0,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10055 entries, 0 to 10054
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   created_at           10055 non-null  object 
 1   d_ad_account_id      10055 non-null  object 
 2   d_utm_source         10055 non-null  object 
 3   d_utm_medium         10055 non-null  object 
 4   d_utm_campaign       10055 non-null  object 
 5   d_utm_content        10055 non-null  object 
 6   m_clicks             10055 non-null  float64
 7   m_cost               10055 non-null  float64
 8   lead_id              2531 non-null   object 
 9   client_id            2531 non-null   object 
 10  purchase_created_at  1458 non-null   object 
 11  purchase_id          1458 non-null   object 
 12  m_purchase_amount    1458 non-null   float64
dtypes: float64(3), object(10)
memory usage: 1.1+ MB


None

Количество явных дубликатов: 0


# Итоговая таблица

In [16]:
# сделаем сводную таблицу с агрегирующими функциями
final = (
    pd.pivot_table(merge_ad_lead_client,
                   index=KEYS,
                   values=['m_clicks',
                           'm_cost',
                           'lead_id',
                           'purchase_id',
                           'm_purchase_amount'],
                   aggfunc={'m_clicks': 'sum',
                            'm_cost': 'sum',
                            'lead_id': 'count',
                            'purchase_id': 'count',
                            'm_purchase_amount': 'sum'})
    .reset_index()
)

In [17]:
# посчитаем нужные метрики
final['cpl_per_day'] = final.m_cost / final.lead_id
final['roas_per_day_perc'] = final.m_purchase_amount / final.m_cost * 100
final.cpl_per_day = final.cpl_per_day.replace(np.inf, 0)
final = final.fillna(0)

In [18]:
final.head()

,created_at,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,lead_id,m_clicks,m_cost,m_purchase_amount,purchase_id,cpl_per_day,roas_per_day_perc
0,2022-01-03,yandex,cpc,48306435,8404700756,0,0.0,0.000,0.0,0,0.0,0.0
1,2022-01-03,yandex,cpc,48306435,8404700757,0,1.0,11.952,0.0,0,0.0,0.0
2,2022-01-03,yandex,cpc,48306435,8813476161,0,0.0,0.000,0.0,0,0.0,0.0
3,2022-01-03,yandex,cpc,48306435,8813476162,0,0.0,0.000,0.0,0,0.0,0.0
4,2022-01-03,yandex,cpc,48306435,8813476163,0,0.0,0.000,0.0,0,0.0,0.0


In [19]:
# финальная подгтовка таблицы к выгрузке
final = final.drop('d_utm_content', axis=1)
final = final.rename(columns={
    'lead_id': 'leads_total',
    'm_clicks': 'clicks_total',
    'm_cost': 'cost_total',
    'm_purchase_amount': 'total_sales_revenue',
    'purchase_id': 'purchases_total'})

In [20]:
# раскомментируйте для загрузки в папку
# final.to_excel(f'final-{datetime.now().date()}.xlsx')

# Код построения пайплайна

```python
# импортирование библиотек
import pandas as pd
import numpy as np
import re
from datetime import datetime

COSTS = 'm_costs'
CLICKS = 'm_clicks'
DATE = 'created_at'
KEYS = ['created_at',
        'd_utm_source',
        'd_utm_medium',
        'd_utm_campaign',
        'd_utm_content']

# загрузка данных
path = '' # укажите путь к файлу данных

ads = pd.read_csv(path + 'ads.csv')
leads = pd.read_csv(path + 'leads.csv')
purchases = pd.read_csv(path + 'purchases.csv')

# обработка логических несоответствий
leads = leads[leads.lead_created_at >= ads.created_at.min()]

# обработка неинформативного столбца 'd_utm_term'
ads = ads.drop('d_utm_term', axis=1).reset_index(drop=True)
leads = leads[leads.d_lead_utm_term.isna()].drop('d_lead_utm_term', axis=1).reset_index(drop=True)

# приведение данных к одному типу
ads[['d_utm_campaign','d_utm_content']] = \
ads[['d_utm_campaign','d_utm_content']].astype(str)

# обработка пропущенных значений
leads = leads.fillna('unknown')
purchases.client_id = purchases.client_id.fillna('unknonw_client')

# унификация названий столбцов для объединения таблиц
for old_col in leads.columns:
    new_col = re.sub('_lead', '', old_col)
    leads = leads.rename(columns={old_col: new_col})
leads = leads.rename(columns={'lead_created_at': DATE})
    
# объединение таблиц
merge_ad_lead = ads.merge(leads,
                          how='left',
                          on=KEYS)

merge_ad_lead_client = merge_ad_lead.merge(purchases,
                                           how='left',
                                           on='client_id')

# сделаем сводную таблицу с агрегирующими функциями
final = (
    pd.pivot_table(merge_ad_lead_client,
                   index=KEYS,
                   values=['m_clicks',
                           'm_cost',
                           'lead_id',
                           'purchase_id',
                           'm_purchase_amount'],
                   aggfunc={'m_clicks': 'sum',
                            'm_cost': 'sum',
                            'lead_id': 'count',
                            'purchase_id': 'count',
                            'm_purchase_amount': 'sum'})
    .reset_index()
)

# посчитаем нужные метрики
final['cpl_per_day'] = final.m_cost / final.lead_id
final['roas_per_day_perc'] = final.m_purchase_amount / final.m_cost * 100
final.cpl_per_day = final.cpl_per_day.replace(np.inf, 0)
final = final.fillna(0)

# финальная подгтовка таблицы к выгрузке
final = final.drop('d_utm_content', axis=1)
final = final.rename(columns={
    'lead_id': 'leads_total',
    'm_clicks': 'clicks_total',
    'm_cost': 'cost_total',
    'm_purchase_amount': 'total_sales_revenue',
    'purchase_id': 'purchases_total'})

# раскомментируйте для загрузки в папку
# final.to_excel(f'final-{datetime.now().date()}.xlsx')
```